In [10]:
import numpy as np
import torch
from torch.autograd import Variable
from matplotlib import pyplot as plt


In [15]:
import numpy as np
import torch
from torch.nn.init import normal, constant
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
from torch.nn import functional as F
from collections import OrderedDict

class Net(nn.Module):
    def __init__(self, n_channel=96, n_hidden=100, n_output=51, dropout=0.9, std=0.01):
        super().__init__()
        self.n_channel = n_channel
        self.std = std
        self.n_output = n_output
        self.dropout = dropout

        if not isinstance(n_hidden, (list, tuple)):
            n_hidden = (n_hidden,)


        # prune out 0
        n_hidden = [i for i in n_hidden if i != 0]

        self.n_hidden = n_hidden

        n_prev = n_channel
        hiddens = []

        for n in n_hidden:
            hiddens.append(nn.Linear(n_prev, n))
            hiddens.append(nn.ReLU())
            if dropout > 0.0:
                hiddens.append(nn.Dropout(p=dropout, inplace=True))
            n_prev = n
        if len(hiddens) > 0:
            self.hiddens = nn.Sequential(*hiddens)
        else:
            self.hiddens = lambda x: x
        self.ro_layer = nn.Linear(n_prev, n_output)

        self.initialize()

    def forward(self, x):
        x = self.hiddens(x)
        x = self.ro_layer(x)
        return x

    def initialize(self):
        def fn(mod):
            if isinstance(mod, nn.Linear):
                normal(mod.weight, std=self.std)
                constant(mod.bias, 0)
        self.apply(fn)

In [5]:
print('Working!')

#bin_width, bin_counts, clip_outside = (BinConfig() & key).fetch1('bin_width', 'bin_counts', 'clip_outside')
bin_counts = 61
bin_width = 1.0
clip_outside = True
delta = bin_width
nbins = bin_counts

Working!


In [8]:
sigmaA = 3
sigmaB = 15
pv = (np.arange(nbins) - nbins//2) * delta
prior = np.log(np.exp(- pv**2 / 2 / sigmaA**2) / sigmaA + np.exp(- pv**2 / 2 / sigmaB**2) / sigmaB)
prior = Variable(torch.from_numpy(prior)).cuda().float()

/tmp/ipykernel_89596/969746929.py:52: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  normal(mod.weight, std=self.std)
/tmp/ipykernel_89596/969746929.py:53: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  constant(mod.bias, 0)


In [ ]:
net = Net(n_output=61, n_hidden=[600, 600], std=0.001, dropout=0.5)
net.cuda()
loss = nn.CrossEntropyLoss().cuda()

net.eval()
y = net(valid_x)

y = y + prior
val, idx = torch.max(y, 1)
yd = y.data.cpu().numpy()

plt.subplot(211)
t_hat = idx.data.cpu().numpy()
plt.scatter(t_hat, valid_ori)

print(np.sqrt(np.mean((t_hat - valid_ori)**2)) * delta)

plt.subplot(212)

plt.plot(yd[47])

## ISSUES

1. Sequential data recording
2. 